# Exemplo Geração de Mapas - OOBr

> O conjunto de dados utilizados neste exemplo possui dimensionalidade de 54 linhas por 28 colunas.


---
**Variáveis Obstétricas:**

*   Nascimentos - Quantidade de Nascimentos
*   Porc_prematuros - Percentual de Prematuros
*   Porc_cesareas - Percentual de Cesáreas
*   Porc_grav_multipla - Percentual de Gravidez Multiplas
*   Porc_anomalias - Percentual de Anomalias
*   Porc_nenhuma_consulta - Percentual de nenhuma consulta
*   Porc_consulta7mais - Percentual de mais de 7 consultas
*   Porc_feminino - Percentual do sexo feminino
*   Porc_raca_mae_branca - Percentual de mãe com raça branca
*   Porc_raca_mae_negra - Percentual de  mãe com raça negra
*   Porc_peso_menor_2500 - Percentual de peso menor de 2.5Kg
*   Porc_apgar1_menor_7 - Percentual de  APGAR1 < 7 
*   Porc_apgar5_menor_7 - Percentual de  APGAR5 < 7
---

**Variáveis Socioeconômicas:**

*   GINI - Mede o grau de desigualdade existente na distribuição de indivíduos segundo a renda domiciliar per capita. Seu valor varia de 0, quando não há desigualdade (a renda domiciliar per capita de todos os indivíduos tem o mesmo valor), a 1, quando a desigualdade é máxima (apenas um indivíduo detém toda a renda).O universo de indivíduos é limitado àqueles que vivem em domicílios particulares permanentes.
*   T_DES - Percentual da população economicamente ativa (PEA) nessa faixa etária que estava desocupada, ou seja, que não estava ocupada na semana anterior à data do Censo mas havia procurado trabalho ao longo do mês anterior à data dessa pesquisa.
*   T_ANALF15M - Taxa de analfabetismo - 15 anos ou mais 
*   PAREDE - Percentual de pessoas em domicílios com paredes inadequadas
*   T_AGUA - Percentual da população em domicílios com água encanada
*   T_BANAGUA - Percentual da população em domicílios com banheiro e água encanada
*   AGUA_ESGOTO - Razão entre as pessoas que vivem em domicílios cujo abastecimento de água não provem de rede geral e cujo esgotamento sanitário não é realizado por rede coletora de esgoto ou fossa séptica e a população total residente em domicílios particulares permanentes multiplicado por 100. São considerados apenas os domicílios particulares permanentes.
*   T_LIXO - Percentual da população em domicílios com coleta de lixo

# Carregamento dos Dados

In [ ]:
import pandas as pd

dados = pd.read_csv("dados_brutos_com_latitudes_obstetricos_ibge.csv", sep=';')

# Instalação da Biblioteca de geração de mapa (plotly)

In [ ]:
!pip install plotly --quiet
!pip install -U plotly --quiet
import plotly
print(plotly.__version__)
import plotly as plty
import plotly.express as px
import json
from urllib.request import urlopen


In [ ]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
        Brazil = json.load(response) # Javascrip object notation 

state_id_map = {}
for feature in Brazil ['features']:
 feature['id'] = feature['properties']['name']
 state_id_map[feature['properties']['sigla']] = feature['id']

# Geração do Mapa

In [ ]:
### Definição da variável padrão
feat = 'Porc_cesareas'

### Estilos de Mapa disponíveis
estil_map = ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd']

'''
Função Para Impressão do Mapa.
'''

def imprime_mapa_by_features_plus(dados, feat, cluster, ano, estilo, animacao):
  #feat = "Nascimentos"

  if cluster != "":
    dados_to_plot = dados.query('Cluster  ==' + str(cluster))
  else:
    dados_to_plot = dados

  if ano != "":
    dados_to_plot = dados_to_plot.query('Ano  ==' + str(ano))


  if animacao == 'Cluster':
    dados_to_plot = dados_to_plot.sort_values(by='Cluster')
  else:
    dados_to_plot = dados_to_plot.sort_values(by='Ano')
  




  if dados_to_plot.shape[0]>0:

    fig = px.choropleth(
                        dados_to_plot, #soybean database
                        locations = 'Estado', #define the limits on the map/geography
                        geojson = Brazil, #shape information
                        color = feat, #defining the color of the scale through the database
                        color_continuous_scale=estilo,
                        hover_name = 'Estado', #the information in the box
                        hover_data =[feat,"Longitude","Latitude"],
                        #title = "OOBr - Dados Socioeconômicos & Obstétricos -  Cluster: " + str(cluster) + " - Ano " + str(ano) , #title of the map
                        title = "OOBr - Dados Socioeconômicos & Obstétricos", #title of the map
                        animation_frame = animacao #creating the application based on the year
                        )
    fig.update_geos(fitbounds = "locations", visible = True)     
    fig.show()
    
    


In [ ]:
#@title Imprime Mapa


ano = "" #@param ["", "2000", "2010"]
variavel = "Nascimentos" #@param ["Nascimentos", "Porc_prematuros", "Porc_cesareas", "Porc_grav_multipla",              "Porc_anomalias", "Porc_nenhuma_consulta", "Porc_consulta7mais",              "Porc_feminino", "Porc_raca_mae_branca", "Porc_raca_mae_negra",              "Porc_peso_menor_2500", "Porc_apgar1_menor_7", "Porc_apgar5_menor_7",              "GINI", "T_DES", "T_ANALF15M", "PAREDE", "T_AGUA", "T_BANAGUA", "AGUA_ESGOTO", "T_LIXO",  "CLUSTER"]
cluster = "" #@param ["", "0", "1", "2", "3", "4", "5"]
animacao = "Ano" #@param ["Ano", "Cluster"]
estilo = "tempo" #@param ["aggrnyl", "agsunset", "algae", "amp", "armyrose", "balance", "blackbody", "bluered", "blues", "blugrn", "bluyl", "brbg",              "brwnyl", "bugn", "bupu", "burg", "burgyl", "cividis", "curl",              "darkmint", "deep", "delta", "dense", "earth", "edge", "electric",              "emrld", "fall", "geyser", "gnbu", "gray", "greens", "greys",              "haline", "hot", "hsv", "ice", "icefire", "inferno", "jet",              "magenta", "magma", "matter", "mint", "mrybm", "mygbm", "oranges",              "orrd", "oryel", "oxy", "peach", "phase", "picnic", "pinkyl",              "piyg", "plasma", "plotly3", "portland", "prgn", "pubu", "pubugn",              "puor", "purd", "purp", "purples", "purpor", "rainbow", "rdbu",              "rdgy", "rdpu", "rdylbu", "rdylgn", "redor", "reds", "solar",              "spectral", "speed", "sunset", "sunsetdark", "teal", "tealgrn",              "tealrose", "tempo", "temps", "thermal", "tropic", "turbid",              "turbo", "twilight", "viridis", "ylgn", "ylgnbu", "ylorbr",              "ylorrd"]


#@markdown ---
imprime_mapa_by_features_plus(dados,variavel, cluster,ano, estilo, animacao )




In [ ]:
import matplotlib.pyplot as plt_
import seaborn as sea


fator = .75
tam_x = 18.16*fator #int(df_clu.shape[1]/10)+16.18
tam_y = 10*fator 
fig, ax = plt_.subplots(figsize=(tam_x,tam_y))
sea.barplot(data=dados.query('Ano ==2010')[['UF', variavel]], x=variavel, y='UF', ax=ax)
plt_.grid(color='grey', linestyle=':', linewidth=.5)
plt_.title("Estados - 2010 - " + variavel)
plt_.show()  
